In [18]:
from transformers import AutoModelForCausalLM 
from transformers import AutoProcessor 

To Download hugging face models locally 

To download a Large Language Model (LLM) from Hugging Face for local inference, follow these steps:

Step 1: Install Hugging Face CLI
Install the Hugging Face CLI:

```bash
pip install huggingface_hub
```

Step 2: Download the Model
Choose the Model: Identify the model you want to download from the Hugging Face Hub. Download the Model: Use the following command to download the model:

```bash
huggingface-cli download THE_MODEL_OWNER/THE_MODEL_NAME
```

Replace THE_MODEL_OWNER and THE_MODEL_NAME with the appropriate values for the model you want to download


In [37]:
model_id = "C:\\Users\\leestott\\.cache\\huggingface\\hub\\models--microsoft--Phi-3.5-vision-instruct\\snapshots\\4a0d683eba9f1d0cbfb6151705d1ee73c25a80ca" 
# change module to Phi3.5 ONNX
# model_id = ""


In [38]:
model = AutoModelForCausalLM.from_pretrained(model_id, device_map="cuda", trust_remote_code=True, torch_dtype="auto", _attn_implementation='eager') # use _attn_implementation='eager' to disable flash attention


Loading checkpoint shards: 100%|██████████| 2/2 [00:24<00:00, 12.15s/it]


In [6]:
processor = AutoProcessor.from_pretrained(model_id, trust_remote_code=True) 

c:\Users\leestott\AppData\Local\miniforge\envs\recycling\lib\site-packages\transformers\models\auto\image_processing_auto.py:517: FutureWarning: The image_processor_class argument is deprecated and will be removed in v4.42. Please use `slow_image_processor_class`, or `fast_image_processor_class` instead
  warnings.warn(


In [7]:
messages = [ 
    {"role": "user", "content": "<|image_1|>\nWhat is shown in this image?"}
] 

In [8]:
from PIL import Image 
import requests 

In [9]:
url = "./demo.png" 
image = Image.open(url) 

In [10]:
prompt = processor.tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)


In [11]:
inputs = processor(prompt, [image], return_tensors="pt").to("cuda:0") 

In [12]:
generation_args = { 
    "max_new_tokens": 500, 
    "temperature": 0.3, 
    "do_sample": False, 
} 

In [13]:
generate_ids = model.generate(**inputs, eos_token_id=processor.tokenizer.eos_token_id, **generation_args) 


c:\Users\leestott\AppData\Local\miniforge\envs\recycling\lib\site-packages\transformers\generation\configuration_utils.py:601: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.3` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The `seen_tokens` attribute is deprecated and will be removed in v4.41. Use the `cache_position` model input instead.
c:\Users\leestott\AppData\Local\miniforge\envs\recycling\lib\site-packages\transformers\models\clip\modeling_clip.py:491: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(
You are not running the flash-attention implementation, expect numerical differences.


In [14]:
generate_ids = generate_ids[:, inputs['input_ids'].shape[1]:]
response = processor.batch_decode(generate_ids, skip_special_tokens=True, clean_up_tokenization_spaces=False)[0] 


In [15]:
response

'The image displays a pie chart illustrating the distribution of marks obtained in an exam across four different subjects: maths, physics, chemistry, and English.'